## Try DuckDB

[https://duckdb.org](https://duckdb.org)

In [1]:
import sys
sys.path

['/Users/mjboothaus/code/github/databooth/try-duckdb',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/try-duckdb/lib/python38.zip',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/try-duckdb/lib/python3.8',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/try-duckdb/lib/python3.8/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/try-duckdb/lib/python3.8/site-packages',
 '/opt/homebrew/Caskroom/mambaforge/base/envs/try-duckdb/lib/python3.8/site-packages/IPython/extensions',
 '/Users/mjboothaus/.ipython']

In [2]:
import duckdb

In [3]:
cursor = duckdb.connect()
print(cursor.execute('SELECT 42').fetchall())

[(42,)]


In [4]:
import pandas as pd

In [5]:
mydf = pd.DataFrame({'a' : [1, 2, 3]})
print(duckdb.query("SELECT AVG(a) FROM mydf").to_df())

   avg(a)
0     2.0


In [6]:
!jupyter kernelspec list

[ListKernelSpecs] WARNING | Config option `kernel_spec_manager_class` not recognized by `ListKernelSpecs`.
Available kernels:
  python3810jvsc74a57bd03086ab3d90372c825407b804b6256634f337f6e28cf9335d0f6d2a4e138be96d    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd03086ab3d90372c825407b804b6256634f337f6e28cf9335d0f6d2a4e138be96d
  python3810jvsc74a57bd099143769d1b812eb68db2dcb5b68ac9d4e914bf3cc41efab59d780dae4e055d1    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd099143769d1b812eb68db2dcb5b68ac9d4e914bf3cc41efab59d780dae4e055d1
  python3810jvsc74a57bd0cfa7a8ce30da2c99891659cad67a24848f787c9950064e6e35407c1189265faa    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd0cfa7a8ce30da2c99891659cad67a24848f787c9950064e6e35407c1189265faa
  python3810jvsc74a57bd0f240be019856415b64564560067a995388062410618151689d60bca2458eef6d    /Users/mjboothaus/Library/Jupyter/kernels/python3810jvsc74a57bd0f240be019856415b64564560067a995388062410618151689d60bca

## Examples

From https://deepnote.com/@abid/Data-Science-with-DuckDB-9KKvj1EoQrmj6nj4Y2prkg

In [7]:
# connect to an in-memory temporary database
conn = duckdb.connect()

# run arbitrary SQL commands
conn.execute("CREATE TABLE test_table (i INTEGER, j STRING)")

In [8]:
conn.execute("INSERT INTO test_table VALUES (1, 'one'),(9,'nine')")

In [10]:
my_df = conn.execute("SELECT i from test_table ").fetchdf()

In [11]:
my_df

,i
0,1
1,9


In [12]:
# we can use placeholders for parameters
conn.execute("INSERT INTO test_table VALUES (?, ?)", [2, 'two'])


conn.executemany("INSERT INTO test_table VALUES (?, ?)", [[3, 'three'], [4, 'four']])

In [13]:
conn.execute("SELECT * from test_table ").fetchdf()

,i,j
0,1,one
1,9,nine
2,2,two
3,3,three
4,4,four


In [14]:
print(conn.execute("SELECT * FROM test_table").fetchnumpy())

{'i': array([1, 9, 2, 3, 4], dtype=int32), 'j': array(['one', 'nine', 'two', 'three', 'four'], dtype=object)}


In [15]:
test_df = pd.DataFrame.from_dict({"i":[1, 2, 3, 4], "j":["one", "two", "three", "four"]})

In [16]:
test_df

,i,j
0,1,one
1,2,two
2,3,three
3,4,four


In [17]:
# make this data frame available as a view in duckdb
conn.register("test_df", test_df)
print(conn.execute("SELECT j FROM test_df WHERE i > 1").fetchdf())

       j
0    two
1  three
2   four


In [18]:
df = pd.read_csv("/Users/mjboothaus/data/kaggle/bank_data.csv")
df.head()

,id,expected_recovery_amount,actual_recovery_amount,recovery_strategy,age,sex
0,2030,194,263.540,Level 0 Recovery,19,Male
1,1150,486,416.090,Level 0 Recovery,25,Female
2,380,527,429.350,Level 0 Recovery,27,Male
3,1838,536,296.990,Level 0 Recovery,25,Male
4,1995,541,346.385,Level 0 Recovery,34,Male


In [19]:
# make this data frame available as a view in duckdb
conn.register("bank_df", df)
conn.execute("SELECT actual_recovery_amount FROM bank_df WHERE age > 27").fetchdf()

,actual_recovery_amount
0,346.385000
1,520.710000
2,221.585000
3,201.815000
4,262.445000
...,...
1434,15388.195990
1435,9709.909257
1436,30209.181790
1437,20386.232190


In [20]:
rel = conn.from_df(test_df)
rel

---------------------
-- Expression Tree --
---------------------
pandas_scan(4967208608)

---------------------
-- Result Columns  --
---------------------
- i (BIGINT)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
BIGINT	VARCHAR	
[ Rows: 4]
1	one	
2	two	
3	three	
4	four	



In [21]:
rel = duckdb.df(test_df)
rel

---------------------
-- Expression Tree --
---------------------
pandas_scan(4967208608)

---------------------
-- Result Columns  --
---------------------
- i (BIGINT)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
BIGINT	VARCHAR	
[ Rows: 4]
1	one	
2	two	
3	three	
4	four	



In [22]:
rel = conn.table("test_table")
rel

---------------------
-- Expression Tree --
---------------------
Scan Table [test_table]

---------------------
-- Result Columns  --
---------------------
- i (INTEGER)
- j (VARCHAR)

---------------------
-- Result Preview  --
---------------------
i	j	
INTEGER	VARCHAR	
[ Rows: 5]
1	one	
9	nine	
2	two	
3	three	
4	four	



In [23]:
rel = duckdb.from_csv_auto("/Users/mjboothaus/data/kaggle/bank_data.csv")
rel

---------------------
-- Expression Tree --
---------------------
read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- id (INTEGER)
- expected_recovery_amount (INTEGER)
- actual_recovery_amount (DOUBLE)
- recovery_strategy (VARCHAR)
- age (INTEGER)
- sex (VARCHAR)

---------------------
-- Result Preview  --
---------------------
id	expected_recovery_amount	actual_recovery_amount	recovery_strategy	age	sex	
INTEGER	INTEGER	DOUBLE	VARCHAR	INTEGER	VARCHAR	
[ Rows: 10]
2030	194	263.540000	Level 0 Recovery	19	Male	
1150	486	416.090000	Level 0 Recovery	25	Female	
380	527	429.350000	Level 0 Recovery	27	Male	
1838	536	296.990000	Level 0 Recovery	25	Male	
1995	541	346.385000	Level 0 Recovery	34	Male	
731	548	520.710000	Level 0 Recovery	35	Male	
221	549	221.585000	Level 0 Recovery	33	Male	
1932	560	373.720000	Level 0 Recovery	19	Female	
1828	562	201.865000	Level 0 Recovery	22	Female	
2001	565	504.885000	Level 0 Recovery	2

In [24]:
rel.alias

'/Users/mjboothaus/data/kaggle/bank_data.csv'

In [25]:
rel.columns

['id',
 'expected_recovery_amount',
 'actual_recovery_amount',
 'recovery_strategy',
 'age',
 'sex']

In [26]:
rel.types

['INTEGER', 'INTEGER', 'DOUBLE', 'VARCHAR', 'INTEGER', 'VARCHAR']

In [27]:
rel.filter('age > 18')

---------------------
-- Expression Tree --
---------------------
Filter [age>18]
  read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- id (INTEGER)
- expected_recovery_amount (INTEGER)
- actual_recovery_amount (DOUBLE)
- recovery_strategy (VARCHAR)
- age (INTEGER)
- sex (VARCHAR)

---------------------
-- Result Preview  --
---------------------
id	expected_recovery_amount	actual_recovery_amount	recovery_strategy	age	sex	
INTEGER	INTEGER	DOUBLE	VARCHAR	INTEGER	VARCHAR	
[ Rows: 10]
2030	194	263.540000	Level 0 Recovery	19	Male	
1150	486	416.090000	Level 0 Recovery	25	Female	
380	527	429.350000	Level 0 Recovery	27	Male	
1838	536	296.990000	Level 0 Recovery	25	Male	
1995	541	346.385000	Level 0 Recovery	34	Male	
731	548	520.710000	Level 0 Recovery	35	Male	
221	549	221.585000	Level 0 Recovery	33	Male	
1932	560	373.720000	Level 0 Recovery	19	Female	
1828	562	201.865000	Level 0 Recovery	22	Female	
2001	565	504.885000	

In [28]:
rel.project('id, age')

---------------------
-- Expression Tree --
---------------------
Projection [id, age]
  read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- id (INTEGER)
- age (INTEGER)

---------------------
-- Result Preview  --
---------------------
id	age	
INTEGER	INTEGER	
[ Rows: 10]
2030	19	
1150	25	
380	27	
1838	25	
1995	34	
731	35	
221	33	
1932	19	
1828	22	
2001	27	



In [29]:
rel.order('sex')

---------------------
-- Expression Tree --
---------------------
Order [sex DESC]
  read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- id (INTEGER)
- expected_recovery_amount (INTEGER)
- actual_recovery_amount (DOUBLE)
- recovery_strategy (VARCHAR)
- age (INTEGER)
- sex (VARCHAR)

---------------------
-- Result Preview  --
---------------------
id	expected_recovery_amount	actual_recovery_amount	recovery_strategy	age	sex	
INTEGER	INTEGER	DOUBLE	VARCHAR	INTEGER	VARCHAR	
[ Rows: 10]
1769	604	278.720000	Level 0 Recovery	32	Female	
1150	486	416.090000	Level 0 Recovery	25	Female	
1932	560	373.720000	Level 0 Recovery	19	Female	
1376	2249	3088.061261	Level 2 Recovery	23	Female	
1028	2253	2871.750943	Level 2 Recovery	25	Female	
1781	9920	20386.232190	Level 4 Recovery	67	Female	
196	9857	9709.909257	Level 4 Recovery	68	Female	
313	9859	30209.181790	Level 4 Recovery	58	Female	
1850	613	377.105000	Level 0 Recovery	28	Fe

In [30]:
rel.limit(2)

---------------------
-- Expression Tree --
---------------------
Limit 2
  read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- id (INTEGER)
- expected_recovery_amount (INTEGER)
- actual_recovery_amount (DOUBLE)
- recovery_strategy (VARCHAR)
- age (INTEGER)
- sex (VARCHAR)

---------------------
-- Result Preview  --
---------------------
id	expected_recovery_amount	actual_recovery_amount	recovery_strategy	age	sex	
INTEGER	INTEGER	DOUBLE	VARCHAR	INTEGER	VARCHAR	
[ Rows: 2]
2030	194	263.540000	Level 0 Recovery	19	Male	
1150	486	416.090000	Level 0 Recovery	25	Female	



In [31]:
rel.filter('age > 19').project('age').order('sex').limit(2)

---------------------
-- Expression Tree --
---------------------
Limit 2
  Order [sex DESC]
    Projection [age]
      Filter [age>19]
        read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- age (INTEGER)

---------------------
-- Result Preview  --
---------------------
age	
INTEGER	
[ Rows: 2]
67	
58	



In [32]:
rel.aggregate("age, sum(actual_recovery_amount)")

---------------------
-- Expression Tree --
---------------------
Aggregate [age, sum(actual_recovery_amount)]
  read_csv_auto(/Users/mjboothaus/data/kaggle/bank_data.csv)

---------------------
-- Result Columns  --
---------------------
- age (INTEGER)
- sum(actual_recovery_amount) (DOUBLE)

---------------------
-- Result Preview  --
---------------------
age	sum(actual_recovery_amount)	
INTEGER	DOUBLE	
[ Rows: 10]
19	52787.712089	
25	72769.342330	
27	67569.292950	
34	109902.427032	
35	115424.466724	
33	138755.807230	
22	46662.153746	
31	92225.534688	
18	39969.573274	
32	110627.466806	



In [33]:
print(duckdb.filter(df, 'age > 1'))
print(duckdb.project(df, 'age +1'))
print(duckdb.order(df, 'sex'))
print(duckdb.limit(df, 2))

---------------------
-- Expression Tree --
---------------------
Filter [age>1]
  pandas_scan(4967209520)

---------------------
-- Result Columns  --
---------------------
- id (BIGINT)
- expected_recovery_amount (BIGINT)
- actual_recovery_amount (DOUBLE)
- recovery_strategy (VARCHAR)
- age (BIGINT)
- sex (VARCHAR)

---------------------
-- Result Preview  --
---------------------
id	expected_recovery_amount	actual_recovery_amount	recovery_strategy	age	sex	
BIGINT	BIGINT	DOUBLE	VARCHAR	BIGINT	VARCHAR	
[ Rows: 10]
2030	194	263.540000	Level 0 Recovery	19	Male	
1150	486	416.090000	Level 0 Recovery	25	Female	
380	527	429.350000	Level 0 Recovery	27	Male	
1838	536	296.990000	Level 0 Recovery	25	Male	
1995	541	346.385000	Level 0 Recovery	34	Male	
731	548	520.710000	Level 0 Recovery	35	Male	
221	549	221.585000	Level 0 Recovery	33	Male	
1932	560	373.720000	Level 0 Recovery	19	Female	
1828	562	201.865000	Level 0 Recovery	22	Female	
2001	565	504.885000	Level 0 Recovery	27	Male	



-------------

In [34]:
# compute the query result from the relation 
res = rel.execute()
print(res)

In [35]:
# res is a query result, you can call fetchdf() or fetchnumpy() or fetchone() on it
print(res.fetchone())

(2030, 194, 263.54, 'Level 0 Recovery', 19, 'Male')


In [37]:
print(res.fetchall());

[]


In [38]:
# convert a relation back to a pandas data frame
rel.to_df()

,id,expected_recovery_amount,actual_recovery_amount,recovery_strategy,age,sex
0,2030,194,263.540000,Level 0 Recovery,19,Male
1,1150,486,416.090000,Level 0 Recovery,25,Female
2,380,527,429.350000,Level 0 Recovery,27,Male
3,1838,536,296.990000,Level 0 Recovery,25,Male
4,1995,541,346.385000,Level 0 Recovery,34,Male
...,...,...,...,...,...,...
1877,361,9785,15388.195990,Level 4 Recovery,65,Female
1878,196,9857,9709.909257,Level 4 Recovery,68,Female
1879,313,9859,30209.181790,Level 4 Recovery,58,Female
1880,1781,9920,20386.232190,Level 4 Recovery,67,Female


In [40]:
complaints = duckdb.from_csv_auto("/Users/mjboothaus/data/CFPB/complaints.csv")

In [42]:
complaints.to_df().head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-06-13,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CAPITAL ONE FINANCIAL CORPORATION,PA,186XX,NaN,Consent not provided,Web,2019-06-13,Closed with explanation,Yes,N/A,3274605
1,2019-04-01,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,19067,NaN,Consent not provided,Web,2019-04-01,Closed with explanation,Yes,N/A,3198225
2,2021-05-28,"Credit reporting, credit repair services, or o...",Credit reporting,Unable to get your credit report or credit score,Problem getting your free annual credit report,NaN,NaN,"EQUIFAX, INC.",WA,98026,NaN,Other,Web,2021-05-28,Closed with explanation,Yes,N/A,4412190
3,2021-05-28,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Credit inquiries on your report that you don't...,NaN,NaN,"EQUIFAX, INC.",AL,35211,NaN,Consent not provided,Web,2021-05-28,Closed with explanation,Yes,N/A,4414038
4,2021-06-16,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,not mine : FCRA violations : see attachments f...,NaN,"EQUIFAX, INC.",TX,773XX,NaN,Consent provided,Web,2021-06-16,Closed with explanation,Yes,N/A,4467932


In [43]:
complaints.types

['DATE',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'DATE',
 'VARCHAR',
 'VARCHAR',
 'VARCHAR',
 'INTEGER']